In [1]:
from cil.framework import DataContainer, BlockDataContainer, Partitioner

# Import libraries
from cil.optimisation.utilities import Sampler
from cil.optimisation.algorithms import SPDHG
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.optimisation.functions import IndicatorBox, BlockFunction, L2NormSquared, MixedL21Norm
 
from cil.io import ZEISSDataReader
 
from cil.processors import Slicer, Binner, TransmissionAbsorptionConverter
 
from cil.plugins.astra.operators import ProjectionOperator
from cil.plugins.ccpi_regularisation.functions import FGP_TV
 
from cil.utilities.display import show2D
 
 
import numpy as np
import matplotlib.pyplot as plt
import os



SyntaxError: keyword argument repeated: update_objective_interval (SPDHG.py, line 125)

In [ ]:
reader = ZEISSDataReader()
filename = '../data/valnut_tomo-A.txrm'
reader.set_up(file_name=filename)
data3D = reader.read()

# reorder data to match default order for Astra/Tigre operator
data3D.reorder('astra')

# Get Image and Acquisition geometries
ag3D = data3D.geometry
ig3D = ag3D.get_ImageGeometry()

num_subsets = 80

partitioned_data=data3D.partition(num_subsets, 'sequential')
show2D(partitioned_data)


# Initialize the lists containing the F_i's and A_i's
f_subsets = []


# Define F_i's 
for i in range(num_subsets):
    # Define F_i and put into list
    fi = 0.5*L2NormSquared(b = partitioned_data[i])
    f_subsets.append(fi)
    
    
ageom_subset = partitioned_data.geometry
A = ProjectionOperator(ig3D, ageom_subset)

#F = L2NormSquared.fromBlockDataContainer(partitioned_data, constant=0.5)

# Define F and K
F = BlockFunction(*f_subsets)
K = A

# Define G (by default the positivity constraint is on)
alpha = 0.025
G = alpha * FGP_TV()

spdhg_sequential = SPDHG(f = F, g = G, operator = K,  max_iteration = 30*num_subsets,
            update_objective_interval = num_subsets, sampler=Sampler.sequential(num_subsets))
spdhg_sequential.run()

In [ ]:



# Extract vertical slice
data2D = data3D.get_slice(vertical='centre')

# Select every 10 angles
sliced_data = Slicer(roi={'angle':(0,1601,10)})(data2D)

# Reduce background regions
binned_data = Binner(roi={'horizontal':(120,-120,2)})(sliced_data)

# Create absorption data 
data = TransmissionAbsorptionConverter()(binned_data) 

# Remove circular artifacts
data -= np.mean(data.as_array()[80:100,0:30])

# Get Image and Acquisition geometries for one slice
ag2D = data.geometry
ag2D.set_angles(ag2D.angles, initial_angle=0.2, angle_unit='radian')
ig2D = ag2D.get_ImageGeometry()

A = ProjectionOperator(ig2D, ag2D, device = "gpu")

#show2D(data)

In [ ]:
import matplotlib
print(matplotlib. __version__)


In [ ]:
# Define number of subsets
num_subsets = 80

partitioned_data=data.partition(num_subsets, 'sequential')
show2D(partitioned_data)


# Initialize the lists containing the F_i's and A_i's
f_subsets = []


# Define F_i's 
for i in range(num_subsets):
    # Define F_i and put into list
    fi = 0.5*L2NormSquared(b = partitioned_data[i])
    f_subsets.append(fi)
    
    
ageom_subset = partitioned_data.geometry
A = ProjectionOperator(ig2D, ageom_subset)

#F = L2NormSquared.fromBlockDataContainer(partitioned_data, constant=0.5)

# Define F and K
F = BlockFunction(*f_subsets)
K = A

# Define G (by default the positivity constraint is on)
alpha = 0.025
G = alpha * FGP_TV()

In [ ]:
print(ageom_subset)
print(A)
print(A.shape[0])

In [ ]:

spdhg_sequential = SPDHG(f = F, g = G, operator = K,  max_iteration = 30*num_subsets,
            update_objective_interval = num_subsets, sampler=Sampler.sequential(num_subsets))
spdhg_sequential.run()

spdhg_recon = spdhg_sequential.solution    

In [ ]:
from numbers import Number
tau=spdhg_sequential._tau
sigma=spdhg_sequential._sigma



In [ ]:
# Setup and run SPDHG for 50 iterations
spdhg_pert = SPDHG(f = F, g = G, operator = K,  max_iteration = 30*num_subsets,
            update_objective_interval = num_subsets, sampler=Sampler.randomWithReplacement(num_subsets))
spdhg_pert.run()

spdhg_recon = spdhg_pert.solution    

In [ ]:
# Setup and run SPDHG for 50 iterations
spdhg_random = SPDHG(f = F, g = G, operator = K,  max_iteration = 30*num_subsets,
            update_objective_interval = num_subsets , sampler=Sampler.randomWithoutReplacement(num_subsets))
spdhg_random.run()

spdhg_recon = spdhg_random.solution    

In [ ]:
spdhg_hermanmeyer = SPDHG(f = F, g = G, operator = K,  max_iteration = 30*num_subsets,
            update_objective_interval = num_subsets, sampler=Sampler.hermanMeyer(num_subsets))
spdhg_hermanmeyer.run()

spdhg_recon = spdhg_hermanmeyer.solution    

In [ ]:
spdhg_staggered = SPDHG(f = F, g = G, operator = K,  max_iteration = 30*num_subsets,
            update_objective_interval = num_subsets, sampler=Sampler.staggered(num_subsets, 3))
spdhg_staggered.run()

spdhg_recon = spdhg_staggered.solution    

In [ ]:
iter_range=range(0,30+1)
plt.figure()
plt.semilogy(iter_range, spdhg_sequential.objective, label='Sequential')
plt.semilogy(iter_range, spdhg_random.objective, label='Random with replacement (default)')
plt.semilogy(iter_range, spdhg_staggered.objective, label='Staggered (offset=3)')
plt.semilogy(iter_range, spdhg_pert.objective, label='Random without replacement')
plt.semilogy(iter_range, spdhg_hermanmeyer.objective, label='Herman Meyer')
plt.xlabel('Epochs')
plt.ylabel('Objective')
plt.ylim(25,60)
plt.legend()
plt.show()